파인튜닝용 데이터셋 생성 실습

In [1]:
import os
import time
import json
from dotenv import load_dotenv
from google import genai
from google.genai import types

from tqdm.notebook import tqdm  # 진행바 표시용
# PDF 처리용
from langchain_community.document_loaders import PyMuPDFLoader 
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [2]:
# 1. 설정
load_dotenv()
api_key = os.environ.get("GEMINI_API_KEY")
client = genai.Client(api_key=api_key)

In [3]:
# 2. PDF 로드 및 분할 (PyMuPDFLoader 사용 - 속도/정확도/메타데이터 확보)
def load_and_split_pdf(file_path):
    print(f"Loading PDF with PyMuPDF: {file_path}...")
    
    # (1) 로드: 페이지별로 텍스트와 메타데이터(페이지번호 등)를 가져옵니다.
    loader = PyMuPDFLoader(file_path)
    docs = loader.load() 
    
    # (2) 분할: 문맥을 고려해 자르되, 페이지 정보 등은 유지합니다.
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000, 
        chunk_overlap=100
    )
    chunks = text_splitter.split_documents(docs) # split_text가 아니라 split_documents 사용
    
    return chunks

In [ ]:
# 3. 데이터셋 생성 루프
def generate_dataset_gemini(chunks, output_file="dataset.jsonl"):
    final_dataset = []
    
    # 프롬프트 설정
    base_prompt = """
    당신은 6G 이동통신과 AI 분야의 전문가입니다. 
    아래 [Context]를 보고 학습용 질문-답변 쌍 3개를 JSON으로 만드세요.
    
    [Context] (Page {page_num})
    {context}
    
    [Format]
    {{ "dataset": [ {{ "instruction": "...", "output": "..." }} ] }}
    """

    print(f" 데이터 생성 시작! (총 {len(chunks)} 단계)")
    
    # 진행바와 함께 반복
    for chunk in tqdm(chunks):
        try:
            # Gemini 1.5 Flash 호출 (JSON 모드 사용)
            response = client.models.generate_content(
                model='gemini-2.5-flash',
                contents=base_prompt.format(
                    context=chunk.page_content, 
                    page_num=chunk.metadata.get('page', 0) + 1
                ),
                config=types.GenerateContentConfig(
                    temperature=0.5,
                    response_mime_type="application/json" 
                )
            )
            
            # 결과 파싱 및 저장
            data = json.loads(response.text)
            if "dataset" in data:
                for item in data["dataset"]:
                    final_dataset.append({
                        "instruction": item["instruction"],
                        "input": "",
                        "output": item["output"]
                    })
            
            time.sleep(3) # API 제한 방지용 잠깐 대기
            
        except Exception as e:
            continue # 에러나면 무시하고 다음으로

    # 파일로 저장
    with open(output_file, "w", encoding="utf-8") as f:
        for entry in final_dataset:
            f.write(json.dumps(entry, ensure_ascii=False) + "\n")
            
    print(f"\n 저장 완료: {output_file} (총 {len(final_dataset)}개 데이터 쌍)")

In [8]:
# PDF 파일 경로
pdf_filename = "src/6g_ai.pdf" 

# 실행
chunks = load_and_split_pdf(pdf_filename)
generate_dataset_gemini(chunks, "6g_ai_dataset.jsonl")

Loading PDF with PyMuPDF: src/6g_ai.pdf...
 데이터 생성 시작! (총 77 단계)


  0%|          | 0/77 [00:00<?, ?it/s]


 저장 완료: 6g_ai_dataset.jsonl (총 0개 데이터 쌍)


데이터 증강

In [6]:
def augment_dataset_gemini(input_file, output_file):
    # 기존 데이터 로드
    with open(input_file, "r", encoding="utf-8") as f:
        original_data = [json.loads(line) for line in f]
    
    augmented_data = []
    
    # 증강 프롬프트 (질문 의미는 같게, 표현은 다르게)
    aug_prompt = """
    아래 주어진 질문(Instruction)과 답변(Output)을 보고, 
    **답변은 그대로 유지하되 질문의 표현만 다르게 바꾼 유사 질문 3개**를 생성하세요.
    
    [Original]
    Q: {instruction}
    A: {output}
    
    [Format]
    {{ "variations": ["유사 질문 1", "유사 질문 2", "유사 질문 3"] }}
    """
    
    print(f" 데이터 증강 시작! (원본 {len(original_data)}개 -> 목표 약 {len(original_data)*4}개)")

    for entry in tqdm(original_data):
        # 원본 데이터는 일단 그대로 추가
        augmented_data.append(entry)
        
        try:
            # Gemini 호출
            response = client.models.generate_content(
                model='gemini-2.5-flash',
                contents=aug_prompt.format(
                    instruction=entry['instruction'], 
                    output=entry['output']
                ),
                config=types.GenerateContentConfig(
                    temperature=0.7, # 다양성을 위해 창의성 약간 높임
                    response_mime_type="application/json"
                )
            )
            
            # 파싱 및 추가
            data = json.loads(response.text)
            if "variations" in data:
                for new_q in data["variations"]:
                    augmented_data.append({
                        "instruction": new_q,     # 새로 만든 질문
                        "input": "",
                        "output": entry['output'] # 답변은 원본 그대로 사용
                    })
            
            time.sleep(3) # 속도 조절
            
        except Exception as e:
            continue

    # 결과 저장
    with open(output_file, "w", encoding="utf-8") as f:
        for entry in augmented_data:
            f.write(json.dumps(entry, ensure_ascii=False) + "\n")
            
    print(f"\n 증강 완료! 총 {len(augmented_data)}개 데이터가 '{output_file}'에 저장됨.")

In [7]:
augment_dataset_gemini("6g_ai_dataset.jsonl", "6g_ai_dataset_augmented.jsonl")

 데이터 증강 시작! (원본 18개 -> 목표 약 72개)


  0%|          | 0/18 [00:00<?, ?it/s]


 증강 완료! 총 27개 데이터가 '6g_ai_dataset_augmented.jsonl'에 저장됨.
